In [1]:
#Import appropriate packages

import robin_stocks as r
import pandas as pd

#Login to Robin Hood API

Username = 'kilian.smith@gmail.com'
Password = 'Fostersmith1993'

Login = r.authentication.login(Username, Password)

In [9]:
#Exports ALL trades, Specify where you want the file stored via the "Folder" input

Folder = 'C:/PYTHON'

Export = r.export.export_completed_stock_orders(Folder,'MY_TRANSACTION_DATA')

#Imports trade file to jupyter notebook

data = pd.read_csv("MY_TRANSACTION_DATA.csv")

#Brings full date and time column down to just the date

data['date'] = data['date'].str[:10]

#Replaces buy/sell column with 1 for buy or -1 for sell

data["side"].replace({"buy": "1", "sell": "-1"}, inplace=True)

#Converts buy/sell column to integar

data["side"] = pd.to_numeric(data["side"])

#multiplies quantity by numeric side column to create positive numbers for buys and negative numbers for sales

data['quantity'] = data[['quantity']].mul(data.side, 0)

#Creates dataframe for symbol, date, and quantity (only fields FactSet needs for buy & hold portfolio)

df = data[['symbol','date','quantity']]

#Flips dataframe

df = df.iloc[::-1]

#Converts date column to datetime

df['date'] = pd.to_datetime(df['date'])

#Takes cumulative sum of trades per symbol as of each transaction data and then pulls forward quantities on 
#each date there was no transation
#Builds out portfolio holdings in format FactSet can accept

dates = pd.date_range(df['date'].min(),df['date'].max(),freq='D')
comb = pd.MultiIndex.from_product((df['symbol'].unique(),dates))

out = df.assign(cum_quant=df.groupby("symbol")['quantity'].cumsum()).merge(
pd.DataFrame(comb.tolist(),columns=['symbol','date']),on=['symbol','date'],how='right')

out = out.assign(quantity=out.sort_values("date").groupby("symbol")['cum_quant'].ffill()
          ).dropna(subset=['quantity']).drop("cum_quant",1)

#Deletes all rows with 0 quantity as those positions have been sold completely

out = out[out.quantity != 0]

#Converts date column from datetime back to date - removes time component

out['date'] = out['date'].dt.date

#Finds duplicate dates for unique symbol (multiple trades placed on single day) and takes the last value

out = out.groupby(['symbol', 'date'])
out = out.last().sort_index().reset_index()

#Exports portfolio data to excel workbook and places in same folder as jupyter notebook is held

out.to_excel("MY_PORTFOLIO_DATA.xlsx",
             sheet_name='MY_PORTFOLIO_DATA', index=False)